### Train a word2vec model using Keras
[reference link](https://www.tensorflow.org/tutorials/text/word2vec)

In [1]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

In [2]:
%load_ext tensorboard
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [3]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=SEED,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [4]:
path_to_file = r"right.txt"
# Use the non empty lines to construct a tf.data.TextLineDataset object for the next step
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [5]:
# lowercase the text and remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 8192
sequence_length = 10

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [6]:
# Call TextVectorization.adapt on the text dataset to create vocabulary
vectorize_layer.adapt(text_ds.batch(1024))

In [7]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'to', 'of', 'and', 'a', 'in', 'that', 'is', 'for', 'on', 'it', 'as', 'he', '—', 'trump', 'with', 'was', 'his']


In [8]:
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [9]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

33712


In [10]:
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[ 99  16   1 166  44  13   1   4   2 238] => ['donald', 'trump', '[UNK]', 'bill', 'clinton', 'as', '[UNK]', 'of', 'the', 'great']
[ 609 1666  482    3  593  155    2 5900 1643   64] => ['open', 'carry', 'comes', 'to', 'texas', 'why', 'the', 'lone', 'star', 'state']
[ 157 1247    1  243  503  558  311  588    3    1] => ['gop', 'field', '[UNK]', 'obama’s', 'move', 'toward', 'executive', 'action', 'to', '[UNK]']
[  46   71  471    3    1  163   46   71    9 5840] => ['president', 'obama', 'wants', 'to', '[UNK]', 'america', 'president', 'obama', 'is', 'plotting']
[   1  433  751    3 1302  178   21 2446 3486 7795] => ['[UNK]', 'family', 'reports', 'to', 'prison', 'does', 'not', 'endorse', 'oregon', 'siege']


In [11]:
# sequences is now a list of int encoded sentences. 
# Just call the generate_training_data function defined earlier to generate training examples for the word2vec model. 
# To recap, the function iterates over each word from each sequence to collect positive and negative context words. 
# Length of target, contexts and labels should be the same, representing the total number of training examples.
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)[:,:,0]
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

100%|██████████| 33712/33712 [02:45<00:00, 203.60it/s]




targets.shape: (186450,)
contexts.shape: (186450, 5)
labels.shape: (186450, 5)


In [12]:
# To perform efficient batching for the potentially large number of training examples, use the tf.data.Dataset API. 
# After this step, you would have a tf.data.Dataset object of (target_word, context_word), (label) elements to train your word2vec model
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<PrefetchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [13]:
num_ns = 4
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

In [14]:
# Compile the model with the tf.keras.optimizers.Adam optimizer
embedding_dim = 512
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [15]:
# define a callback to log training statistics for Tensorboard
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [16]:
# Train the model on the dataset for some number of epochs:
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20
182/182 [==============================] - 8s 13ms/step - loss: 1.4964 - accuracy: 0.4977
Epoch 2/20
182/182 [==============================] - 2s 13ms/step - loss: 1.1231 - accuracy: 0.6597
Epoch 3/20
182/182 [==============================] - 2s 12ms/step - loss: 0.8191 - accuracy: 0.7620
Epoch 4/20
182/182 [==============================] - 2s 11ms/step - loss: 0.6052 - accuracy: 0.8316
Epoch 5/20
182/182 [==============================] - 2s 11ms/step - loss: 0.4643 - accuracy: 0.8778
Epoch 6/20
182/182 [==============================] - 2s 10ms/step - loss: 0.3705 - accuracy: 0.9072
Epoch 7/20
182/182 [==============================] - 2s 11ms/step - loss: 0.3059 - accuracy: 0.9256
Epoch 8/20
182/182 [==============================] - 2s 10ms/step - loss: 0.2602 - accuracy: 0.9377
Epoch 9/20
182/182 [==============================] - 2s 11ms/step - loss: 0.2269 - accuracy: 0.9458
Epoch 10/20
182/182 [==============================] - 2s 11ms/step - loss: 0.2023 - accura

In [17]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [18]:
vocab.index('trump')

16

In [19]:
weights[vocab.index('trump')]

array([ 0.04183096, -0.14334643,  0.08857907, -0.20667455, -0.1791839 ,
        0.02745217, -0.07978299, -0.19490163,  0.20630714,  0.01948943,
       -0.18460403, -0.1077231 ,  0.2540952 , -0.04908137,  0.07910471,
        0.04107846, -0.0042479 , -0.00710103,  0.02440254, -0.27768475,
        0.08844206,  0.33321202, -0.22374986,  0.03017825,  0.16559571,
       -0.25876373, -0.15889056,  0.4555668 , -0.20036584,  0.00602072,
        0.02155547, -0.11949037,  0.2183996 ,  0.09545062, -0.14154981,
       -0.02461464,  0.2817713 ,  0.08902241, -0.17498803,  0.15320791,
       -0.10299405, -0.05487236, -0.04593365,  0.09375723, -0.13382937,
        0.1609062 , -0.10219578, -0.19882713,  0.15592232, -0.00599622,
       -0.31998628,  0.37073052, -0.128623  , -0.02695661, -0.08180618,
        0.10873813, -0.06596952,  0.36303002,  0.05935183,  0.11171316,
       -0.18485136, -0.02479523, -0.1867881 , -0.24616995, -0.04175721,
       -0.09527916,  0.08593529,  0.31076223,  0.18801558,  0.17

In [20]:
# create and save the vectors and metadata files
out_v = io.open('right_vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('right_metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()